In [8]:
%reset -f

import os, sys
sys.path.insert(0, os.path.abspath(".."))

import jax.numpy as jnp

from _utils import plot

2023-01-17 20:05:29.629344: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-17 20:05:29.936367: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-17 20:05:29.947272: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [9]:
## A simple rectangle regularly discretised
class Grid():
    def __init__(self, Lx, Ly, Nx, Ny) -> None:
        self.Lx = Lx
        self.Ly = Ly
        self.Nx = Nx
        self.Ny = Ny

In [10]:
## Reshapes a vector into a Julia-ready matrix for representation
def vec2mat(U, N, M):
    return jnp.reshape(U, (N,M))

## Flattens a matrix into a Julia-ready vector for computation
def mat2vec(u):
    return jnp.reshape(u, (jnp.size(u), -1), order='C')

## Arranges a Julia-ready matrix into a 2D tensor suitable for human visualisation
def mat2plane(u):
    return jnp.flip(u.T)

In [11]:
## A struct to hold out Poisson problem attributes
class Poisson():
    def __init__(self, fval, g) -> None:
        self.g = g
        self.f = jnp.fill(fval, (g.Nx+1,g.Ny+1))
        self.u = jnp.zeros((g.Nx, g.Ny))      ## Boundary conditions are already imposed.
    
    ## Sets a boundary condition all around a matrix
    def setboundaryconditions(self, bc):
        ### Things should be stacked rather than modified
        # u[1,:] .= bc
        # u[g.Nx+1,:] .= bc
        # u[:, 1] .= bc
        # u[:, g.Ny+1] .= bc
        return self.u


In [12]:
## Deternimes location of a given point in a rectangle geometry
## N.B. Assumes matrix form (Ny,Nx), not 2D plane form (Nx,Ny) obtained by rotation
def isonboundary(i:jnp.int32, j:jnp.int32, g:Grid) -> str:
    if j==1:
        return "west"
    elif i==g.Nx+1:
        return "south"
    elif j==g.Ny+1:
        return "east"
    elif i==1:
        return "north"
    else:
        return "inside"